In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('cardekho_dataset.csv', index_col=[0])

In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


#### Data Cleaning

In [4]:
## Check Null Values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
## Remove Unnecesary Columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [8]:
## Getting All Different Types OF Features

num_features = [feature for feature in df.columns if df[feature].dtype !='O']
print("Num of Numerical Feature:", len(num_features))

cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print("Num of Categorical Feature :", len(cat_features))

discrete_features =[feature for feature in num_features if len(df[feature].unique())<=25]
print("Num of Descrete Feature :", len(discrete_features))

continues_features =[feature for feature in num_features if feature not in discrete_features]
print("Num Of Continues Feature :", len(continues_features))

Num of Numerical Feature: 7
Num of Categorical Feature : 4
Num of Descrete Feature : 2
Num Of Continues Feature : 5


In [9]:
## Independent and dependent features

from sklearn.model_selection import train_test_split
X =df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [10]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


#### Feature Encoding ANd Scalling

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [12]:
## Create column transformer with 3 types of transformer

num_features =X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)

    ],remainder='passthrough'
)

In [13]:
## Seprate parameter into train test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
X_train.shape, X_test.shape

((12328, 10), (3083, 10))

#### Model Training Model Selection

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [16]:
## Create a Function to Evaluate the Model

def evaluate_model(true, predicted):
    mae = mean_squared_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_square(true,predicted)
    return mae, rmse, r2_square

In [17]:
## Beginning Model Training
from sklearn.pipeline import Pipeline

models = {
    "Linear Regression":LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "Adaboost":AdaBoostRegressor()
}

# Dictionary to store model pipelines
model_pipelines = {}

# Create pipeline for each model
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),  # same preprocessor used
        ('model', model)
    ])
    model_pipelines[name] = pipeline

from sklearn.metrics import r2_score

for name, pipeline in model_pipelines.items():
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)
    score = r2_score(y_test, preds)
    print(f"{name}: R² Score = {score:.4f}")
    print("---------------------------------")



    y_train_pred =pipeline.predict(X_train)
    y_test_pred =pipeline.predict(X_test)


Linear Regression: R² Score = 0.6645
---------------------------------
Lasso: R² Score = 0.6645
---------------------------------
Ridge: R² Score = 0.6645
---------------------------------
K-Neighbors Regressor: R² Score = 0.9117
---------------------------------
Decision Tree: R² Score = 0.8795
---------------------------------
Random Forest Regressor: R² Score = 0.9307
---------------------------------
Adaboost: R² Score = 0.6773
---------------------------------


In [18]:
knn_params = {"n_neighbors": [2,3,10,20,40 ,50]}
rf_params = {
    "max_depth": [5,8,15,None, 8],
    "max_features": [5,7,"auto", 8],
    "min_samples_split": [2,8,15,20],
    "n_estimators": [100,200,500,1000]
}

ada_params = {
    "n_estimators":[50,60,70,80],
    "loss":['linear','square','exponential']
}

In [19]:
## MOdels LIst for Hyperparamete Tunning
randomcv_models = [
    ('KNN', KNeighborsRegressor(),knn_params),
    ('RF',RandomForestRegressor(),rf_params),
    ('Adaboost',AdaBoostRegressor(),ada_params)
]

In [20]:
## Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}

for name,model,params in randomcv_models:
    random=RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=1
    )

    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"-------------------Best Params for {model_name} ----------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END .....................................n_neighbors=10; total time=   0.0s
[CV] END .....................................n_neighbors=10; total time=   0.0s
[CV] END .....................................n_neighbors=10; total time=   0.0s
[CV] END .....................................n_neighbors=20; total time=   0.0s
[CV] END .....................................n_neighbors=20; total time=   0.0s
[CV] END .....................................n_n

ValueError: 
All the 18 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "d:\All-Machine_Learning_Algorithem-Implimentation\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\All-Machine_Learning_Algorithem-Implimentation\venv\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\All-Machine_Learning_Algorithem-Implimentation\venv\Lib\site-packages\sklearn\neighbors\_regression.py", line 222, in fit
    return self._fit(X, y)
           ^^^^^^^^^^^^^^^
  File "d:\All-Machine_Learning_Algorit